In [1]:
import os
from experiments.data_loaders.standard_loader import MyDataLoader
# from infers.simple_mnist_infer import SimpleMnistInfer
from perception.models.dense_unet import  SegmentionModel
from perception.trainers.segmention_trainer import SegmentionTrainer
from configs.utils.config_utils import process_config
import numpy as np
import cv2
import matplotlib.pyplot as plt

#print(help(plt.imread))

Using TensorFlow backend.


In [11]:
def main_train():
    """
    训练模型

    :return:
    """
    print('[INFO] Reading Configs...')

    config = None

    try:
        config = process_config('./configs/segmention_config.json')  
        config.prob_path = '.\\experiments\\VesselNet\\dataset\\DRIVE\\train\\probmap2\\'
        config.prob_val_path = '.\\experiments\\VesselNet\\dataset\\DRIVE\\validate\\probmap2\\'        
        
        #config.train_img_path = ".\\experiments\\VesselNet\\dataset\\DRIVE\\train\\20_images\\"
        #config.train_groundtruth_path = ".\\experiments\\VesselNet\\dataset\\DRIVE\\train\\final_gt\\"
        #config.seg_num = 4
        #config.num_seg_class = 4
        #config.val_img_path = ".\\experiments\\VesselNet\\dataset\\DRIVE\\validate\\dark\\"
        #config.val_groundtruth_path = ".\\experiments\\VesselNet\\dataset\\DRIVE\\validate\\final_gt\\"
         
     #   config.train_img_path = ".\\experiments\\VesselNet\\dataset\\DRIVE\\train\\bright\\"
     #   config.train_groundtruth_path = ".\\experiments\\VesselNet\\dataset\\DRIVE\\train\\bright_gt\\"
     #   config.val_img_path = ".\\experiments\\VesselNet\\dataset\\DRIVE\\validate\\bright\\"
     #   config.val_groundtruth_path = ".\\experiments\\VesselNet\\dataset\\DRIVE\\validate\\bright_gt\\"
         
        
        #config.train_img_path = ".\\experiments\\VesselNet\\dataset\\HRF\\train\\images\\"
        #config.train_groundtruth_path = ".\\experiments\\VesselNet\\dataset\\HRF\\train\\gt\\"
        #config.val_img_path = ".\\experiments\\VesselNet\\dataset\\HRF\\validate\\images\\"
        #config.val_groundtruth_path = ".\\experiments\\VesselNet\\dataset\\HRF\\validate\\gt\\"
        #config.height = 1168
        #config.width = 1727
        
        #config.val_img_path = ".\\experiments\\VesselNet\\dataset\\DRIVE\\validate\\FOV\\"
        #config.val_groundtruth_path = ".\\experiments\\VesselNet\\dataset\\DRIVE\\validate\\FOV_gt\\"


        
        config.patch_height = 128
        config.patch_width = 128
        config.stride_width = 32
        config.stride_height = 32
        config.train_img_path = ".\\experiments\\VesselNet\\dataset\\DRIVE\\train\\origin\\"
        config.train_groundtruth_path = ".\\experiments\\VesselNet\\dataset\\DRIVE\\train\\gt\\"
        config.val_img_path = ".\\experiments\\VesselNet\\dataset\\DRIVE\\validate\\origin\\"
        config.val_groundtruth_path = ".\\experiments\\VesselNet\\dataset\\DRIVE\\validate\\groundtruth\\"
        #config.height = 540
        #config.width = 540
        
        #config = process_config('./configs/segmention_config_CHASEDB1.json')
       # config = process_config('./configs/segmention_config.json')        
       # config.train_img_path = ".\\experiments\\VesselNet\\dataset\\CHASEDB1\\train\\"
        #config.train_groundtruth_path =".\\experiments\\VesselNet\\dataset\\train\\"
        
        #config.val_img_path = ".\\experiments\\VesselNet\\dataset\\CHASEDB1\\validate\\"
        #config.val_groundtruth_path = ".\\experiments\\VesselNet\\dataset\\validate\\groundtruth\\"
        
        #config.val_datatype = 'jpg'
        #config.val_gt_datatype = 'png'
        #config.height  = 960
        #config.width = 999
        
        
       # config = process_config('./configs/segmention_config_STARE.json')
       #config = process_config('./configs/segmention_config.json')        
       # config.train_img_path = ".\\experiments\\VesselNet\\dataset\\STARE\\img_bmp\\"
       # config.train_groundtruth_path =".\\experiments\\VesselNet\\dataset\\STARE\\gt_bmp\\"
      #  config.val_img_path = ".\\experiments\\VesselNet\\dataset\\STARE\\img_bmp\\"
      #  config.val_groundtruth_path = ".\\experiments\\VesselNet\\dataset\\STARE\\gt_bmp\\"     
      #  config.height = 
      #  config.width = 700
        config.epochs = 15
        
       # config.test_img_path =".\\experiments\\VesselNet\\test\\probmap\\"
        config.epoch = 30
        config.G = 0.8
        config.R = 0.2
        config.B = 0.0
        config.color_space = 'RGB'
        config.batch_size = 8
        #config.color_space = 'HSV'
        config.multi_proportion = 1
        print(config)
    except Exception as e:
        print('[Exception] Config Error, %s' % e)
        exit(0)
    # np.random.seed(47)  # 固定随机数

    print('[INFO] Preparing Data...')
    #dataloader = MyDataLoader(config=config,dataset='CHASEDB1')
    #dataloader = MyDataLoader(config=config,dataset='HRF')
    dataloader = MyDataLoader(config=config,dataset='DRIVE')
   #dataloader = MyDataLoader(config=config,dataset='DRIVE_MULTICLASS')
    #dataloader = MyDataLoader(config=config,dataset='CHASEDB1')
    dataloader.prepare_dataset()
    mode = 'DRIVE'
    #target_path = './metric/STARE/'
    if mode == 'STARE':
        config = process_config('./configs/segmention_config_STARE.json')
        config = process_config('./configs/segmention_config.json')        
        config.train_img_path = ".\\experiments\\VesselNet\\dataset\\STARE\\img_bmp\\"
        config.train_groundtruth_path =".\\experiments\\VesselNet\\dataset\\STARE\\gt_bmp\\"
        config.val_img_path = ".\\experiments\\VesselNet\\dataset\\STARE\\img_bmp\\"
        config.val_groundtruth_path = ".\\experiments\\VesselNet\\dataset\\STARE\\gt_bmp\\"     
        config.height = 605
        config.width = 700
        config.epochs = 6   
        config.color_space = 'LAB'
        #config.color_space = 'HSV'
        config.multi_proportion = False      
        total_imgs,total_gt=dataloader.get_train_data()
        #print(train_imgs.shape) 
        
        for iter_ in range(20):
            if not os.path.exists(target_path+'test'):
                os.mkdir(target_path+'test')
            if not os.path.exists(target_path+'mask'):
                os.mkdir(target_path+'mask')
                
            val_imgs, val_gt = total_imgs[iter_,:,:,:] , total_gt[iter_,:,:,:]
            cv2.imwrite(target_path+'test/'+str(iter_)+'.bmp',val_imgs)
            cv2.imwrite(target_path+'tmask/'+str(iter_)+'.bmp',val_gt)
            val_imgs = np.expand_dims(val_imgs,0)
            val_gt = np.expand_dims(val_gt,0)
            if iter_ == 0:
                
           # val_imgs, val_gt = total_imgs[iter_] , total_gt[iter_]
                train_imgs , train_gt = total_imgs[1:,:,:,:], total_gt[1:,:,:,:]
                #train_imgs, train_gt = np.expand_dims(train_imgs,0),np.expand_dims(train_gt,0)
            elif iter_ == 19:
                train_imgs,train_gt = total_imgs[:iter_,:,:,:], total_gt[:iter_,:,:,:] 
            else:
                train_imgs,train_gt = np.concatenate([total_imgs[:iter_,:,:,:],total_imgs[iter_+1:,:,:,:]],axis=0),np.concatenate([total_gt[:iter_,:,:,:],total_gt[iter_+1:,:,:,:]],axis=0)
            print(iter_)
            print(train_imgs.shape)
            print(total_imgs.shape)
            print('[INFO] Building Model...')
            model = SegmentionModel(config=config)
#
            print('[INFO] Training...iter:{}'.format(iter_))
            trainer = SegmentionTrainer(
            model=model.model,
            data=[train_imgs,train_gt,val_imgs,val_gt],
            config=config,
            iter_=iter_)
            trainer.train()
            print('[INFO] Finishing...iter:{}'.format(iter_))
    else:
        iter_ = 'DRIVE_Binary'
        #total_imgs, total_gt = dataloader.get_train_data()
        #train_imgs = np.array(list(total_imgs[:18,:,:,:]) + list(total_imgs[20:20+18,:,:,:]))
        #train_gt = np.array(list(total_gt[:18,:,:,:]) + list(total_gt[20:20+18,:,:,:]))
        #val_imgs = np.array(list(total_imgs[18:20,:,:,:]) + list(total_imgs[20+18:,:,:,:]))
        #val_gt = np.array(list(total_gt[18:20,:,:,:]) + list(total_gt[20+18:,:,:,:]))        
        #val_imgs = train_imgs[16:20,:,:,:]+train_imgs[20+16:,:,:,:]
        #val_gt = train_gt[16:20,:,:,:]+train_gt[20+16:,:,:,:]
        train_imgs, train_gt = dataloader.get_train_data()

        val_imgs,val_gt= dataloader.get_val_data()
        #prob_map,prob_val_map = dataloader.get_prob_data()
        print(train_imgs.shape)
        #print(val_imgs.shape)
       # print(prob_map.shape)
       # print(prob_val_map.shape)
        print('[INFO] Building Model...')
        model = SegmentionModel(config=config)
        
        print('[INFO] Training...')
        trainer = SegmentionTrainer(
        model=model.model,
        data=[train_imgs,train_gt,val_imgs,val_gt],
        config=config,
        iter_=iter_)
        trainer.train()
        print('[INFO] Finishing...')

In [12]:

if __name__ == '__main__':
    print('Staring Train')
    main_train()
    # test_main()

Staring Train
[INFO] Reading Configs...
B: 0.0
G: 0.8
R: 0.2
batch_size: 8
checkpoint: ./experiments\VesselNet\checkpoint/
color_space: RGB
epoch: 30
epochs: 15
exp_name: VesselNet
hdf5_path: ./experiments\VesselNet\hdf5/
height: 584
multi_proportion: 1
patch_height: 128
patch_width: 128
prob_path: .\experiments\VesselNet\dataset\DRIVE\train\probmap2\
prob_val_path: .\experiments\VesselNet\dataset\DRIVE\validate\probmap2\
seg_num: 1
stride_height: 32
stride_width: 32
subsample: 500
test_datatype: tif
test_gt_datatype: tif
test_gt_path: ./experiments\VesselNet\test/groundtruth/
test_img_path: ./experiments\VesselNet\test/origin/
test_result_path: ./experiments\VesselNet\test/result/
total_train: 40
total_val: 20
train_datatype: tif
train_groundtruth_path: .\experiments\VesselNet\dataset\DRIVE\train\gt\
train_gt_datatype: tif
train_img_path: .\experiments\VesselNet\dataset\DRIVE\train\origin\
val_datatype: tif
val_groundtruth_path: .\experiments\VesselNet\dataset\DRIVE\validate\groundtru

100/100 [==============================] - ETA: 12:42 - loss: 0.7289 - binary_accuracy: 0.5517 - dice_coef: 0.28 - ETA: 6:25 - loss: 0.6944 - binary_accuracy: 0.6130 - dice_coef: 0.3657 - ETA: 4:18 - loss: 0.6837 - binary_accuracy: 0.6128 - dice_coef: 0.374 - ETA: 3:15 - loss: 0.6683 - binary_accuracy: 0.6272 - dice_coef: 0.391 - ETA: 2:36 - loss: 0.6470 - binary_accuracy: 0.6544 - dice_coef: 0.419 - ETA: 2:11 - loss: 0.6304 - binary_accuracy: 0.6775 - dice_coef: 0.445 - ETA: 1:53 - loss: 0.6147 - binary_accuracy: 0.6946 - dice_coef: 0.466 - ETA: 1:39 - loss: 0.6047 - binary_accuracy: 0.7077 - dice_coef: 0.479 - ETA: 1:28 - loss: 0.5927 - binary_accuracy: 0.7177 - dice_coef: 0.494 - ETA: 1:20 - loss: 0.5868 - binary_accuracy: 0.7262 - dice_coef: 0.501 - ETA: 1:13 - loss: 0.5729 - binary_accuracy: 0.7378 - dice_coef: 0.520 - ETA: 1:07 - loss: 0.5615 - binary_accuracy: 0.7470 - dice_coef: 0.535 - ETA: 1:02 - loss: 0.5630 - binary_accuracy: 0.7548 - dice_coef: 0.537 - ETA: 57s - loss: 0.5

100/100 [==============================] - ETA: 14s - loss: 0.2393 - binary_accuracy: 0.9201 - dice_coef: 0.82 - ETA: 13s - loss: 0.2815 - binary_accuracy: 0.9126 - dice_coef: 0.78 - ETA: 13s - loss: 0.2807 - binary_accuracy: 0.9078 - dice_coef: 0.77 - ETA: 13s - loss: 0.2776 - binary_accuracy: 0.9007 - dice_coef: 0.77 - ETA: 12s - loss: 0.2782 - binary_accuracy: 0.9042 - dice_coef: 0.77 - ETA: 12s - loss: 0.2781 - binary_accuracy: 0.9023 - dice_coef: 0.77 - ETA: 12s - loss: 0.2747 - binary_accuracy: 0.9007 - dice_coef: 0.78 - ETA: 12s - loss: 0.2775 - binary_accuracy: 0.9044 - dice_coef: 0.78 - ETA: 12s - loss: 0.2767 - binary_accuracy: 0.9050 - dice_coef: 0.78 - ETA: 12s - loss: 0.2754 - binary_accuracy: 0.9064 - dice_coef: 0.78 - ETA: 11s - loss: 0.2723 - binary_accuracy: 0.9075 - dice_coef: 0.78 - ETA: 11s - loss: 0.2721 - binary_accuracy: 0.9093 - dice_coef: 0.78 - ETA: 11s - loss: 0.2758 - binary_accuracy: 0.9112 - dice_coef: 0.78 - ETA: 11s - loss: 0.2760 - binary_accuracy: 0.91

100/100 [==============================] - ETA: 13s - loss: 0.1908 - binary_accuracy: 0.9388 - dice_coef: 0.84 - ETA: 12s - loss: 0.1951 - binary_accuracy: 0.9256 - dice_coef: 0.83 - ETA: 13s - loss: 0.1959 - binary_accuracy: 0.9159 - dice_coef: 0.83 - ETA: 13s - loss: 0.2156 - binary_accuracy: 0.9191 - dice_coef: 0.81 - ETA: 13s - loss: 0.2265 - binary_accuracy: 0.9232 - dice_coef: 0.81 - ETA: 12s - loss: 0.2247 - binary_accuracy: 0.9235 - dice_coef: 0.81 - ETA: 12s - loss: 0.2307 - binary_accuracy: 0.9222 - dice_coef: 0.80 - ETA: 12s - loss: 0.2287 - binary_accuracy: 0.9245 - dice_coef: 0.80 - ETA: 12s - loss: 0.2286 - binary_accuracy: 0.9239 - dice_coef: 0.80 - ETA: 12s - loss: 0.2312 - binary_accuracy: 0.9244 - dice_coef: 0.80 - ETA: 12s - loss: 0.2290 - binary_accuracy: 0.9253 - dice_coef: 0.80 - ETA: 11s - loss: 0.2336 - binary_accuracy: 0.9253 - dice_coef: 0.80 - ETA: 11s - loss: 0.2340 - binary_accuracy: 0.9257 - dice_coef: 0.80 - ETA: 11s - loss: 0.2338 - binary_accuracy: 0.92

100/100 [==============================] - ETA: 13s - loss: 0.2123 - binary_accuracy: 0.9053 - dice_coef: 0.80 - ETA: 12s - loss: 0.2426 - binary_accuracy: 0.8943 - dice_coef: 0.77 - ETA: 12s - loss: 0.2523 - binary_accuracy: 0.9020 - dice_coef: 0.76 - ETA: 12s - loss: 0.2364 - binary_accuracy: 0.9053 - dice_coef: 0.77 - ETA: 12s - loss: 0.2307 - binary_accuracy: 0.9093 - dice_coef: 0.78 - ETA: 12s - loss: 0.2221 - binary_accuracy: 0.9105 - dice_coef: 0.79 - ETA: 12s - loss: 0.2189 - binary_accuracy: 0.9133 - dice_coef: 0.79 - ETA: 12s - loss: 0.2145 - binary_accuracy: 0.9167 - dice_coef: 0.80 - ETA: 12s - loss: 0.2121 - binary_accuracy: 0.9170 - dice_coef: 0.80 - ETA: 12s - loss: 0.2115 - binary_accuracy: 0.9189 - dice_coef: 0.80 - ETA: 11s - loss: 0.2125 - binary_accuracy: 0.9193 - dice_coef: 0.80 - ETA: 11s - loss: 0.2107 - binary_accuracy: 0.9194 - dice_coef: 0.80 - ETA: 11s - loss: 0.2099 - binary_accuracy: 0.9207 - dice_coef: 0.80 - ETA: 11s - loss: 0.2089 - binary_accuracy: 0.92

...

100/100 [==============================] - ETA: 13s - loss: 0.1866 - binary_accuracy: 0.9225 - dice_coef: 0.81 - ETA: 13s - loss: 0.1873 - binary_accuracy: 0.9157 - dice_coef: 0.81 - ETA: 12s - loss: 0.1822 - binary_accuracy: 0.9223 - dice_coef: 0.82 - ETA: 12s - loss: 0.1842 - binary_accuracy: 0.9255 - dice_coef: 0.81 - ETA: 12s - loss: 0.1924 - binary_accuracy: 0.9289 - dice_coef: 0.81 - ETA: 12s - loss: 0.1887 - binary_accuracy: 0.9274 - dice_coef: 0.81 - ETA: 12s - loss: 0.1863 - binary_accuracy: 0.9273 - dice_coef: 0.81 - ETA: 12s - loss: 0.1834 - binary_accuracy: 0.9296 - dice_coef: 0.81 - ETA: 12s - loss: 0.1813 - binary_accuracy: 0.9316 - dice_coef: 0.82 - ETA: 12s - loss: 0.1788 - binary_accuracy: 0.9299 - dice_coef: 0.82 - ETA: 12s - loss: 0.1761 - binary_accuracy: 0.9301 - dice_coef: 0.82 - ETA: 11s - loss: 0.1771 - binary_accuracy: 0.9277 - dice_coef: 0.82 - ETA: 11s - loss: 0.1779 - binary_accuracy: 0.9272 - dice_coef: 0.82 - ETA: 11s - loss: 0.1770 - binary_accuracy: 0.92

100/100 [==============================] - ETA: 13s - loss: 0.1680 - binary_accuracy: 0.9361 - dice_coef: 0.83 - ETA: 13s - loss: 0.1638 - binary_accuracy: 0.9240 - dice_coef: 0.83 - ETA: 13s - loss: 0.1636 - binary_accuracy: 0.9240 - dice_coef: 0.83 - ETA: 12s - loss: 0.1695 - binary_accuracy: 0.9167 - dice_coef: 0.83 - ETA: 12s - loss: 0.1689 - binary_accuracy: 0.9205 - dice_coef: 0.83 - ETA: 12s - loss: 0.1689 - binary_accuracy: 0.9254 - dice_coef: 0.83 - ETA: 12s - loss: 0.1724 - binary_accuracy: 0.9223 - dice_coef: 0.82 - ETA: 12s - loss: 0.1769 - binary_accuracy: 0.9220 - dice_coef: 0.82 - ETA: 12s - loss: 0.1769 - binary_accuracy: 0.9228 - dice_coef: 0.82 - ETA: 12s - loss: 0.1773 - binary_accuracy: 0.9245 - dice_coef: 0.82 - ETA: 12s - loss: 0.1758 - binary_accuracy: 0.9262 - dice_coef: 0.82 - ETA: 11s - loss: 0.1794 - binary_accuracy: 0.9247 - dice_coef: 0.82 - ETA: 11s - loss: 0.1880 - binary_accuracy: 0.9246 - dice_coef: 0.81 - ETA: 11s - loss: 0.1842 - binary_accuracy: 0.92

100/100 [==============================] - ETA: 13s - loss: 0.1729 - binary_accuracy: 0.9240 - dice_coef: 0.82 - ETA: 13s - loss: 0.1834 - binary_accuracy: 0.9206 - dice_coef: 0.81 - ETA: 13s - loss: 0.1725 - binary_accuracy: 0.9304 - dice_coef: 0.82 - ETA: 13s - loss: 0.1737 - binary_accuracy: 0.9330 - dice_coef: 0.82 - ETA: 13s - loss: 0.1723 - binary_accuracy: 0.9309 - dice_coef: 0.82 - ETA: 12s - loss: 0.1709 - binary_accuracy: 0.9284 - dice_coef: 0.83 - ETA: 12s - loss: 0.1676 - binary_accuracy: 0.9280 - dice_coef: 0.83 - ETA: 12s - loss: 0.1676 - binary_accuracy: 0.9289 - dice_coef: 0.83 - ETA: 12s - loss: 0.1674 - binary_accuracy: 0.9274 - dice_coef: 0.83 - ETA: 12s - loss: 0.1720 - binary_accuracy: 0.9280 - dice_coef: 0.82 - ETA: 12s - loss: 0.1710 - binary_accuracy: 0.9271 - dice_coef: 0.83 - ETA: 11s - loss: 0.1714 - binary_accuracy: 0.9261 - dice_coef: 0.83 - ETA: 11s - loss: 0.1690 - binary_accuracy: 0.9258 - dice_coef: 0.83 - ETA: 11s - loss: 0.1708 - binary_accuracy: 0.92

100/100 [==============================] - ETA: 13s - loss: 0.1424 - binary_accuracy: 0.9508 - dice_coef: 0.85 - ETA: 13s - loss: 0.1497 - binary_accuracy: 0.9349 - dice_coef: 0.85 - ETA: 13s - loss: 0.1534 - binary_accuracy: 0.9266 - dice_coef: 0.84 - ETA: 12s - loss: 0.1530 - binary_accuracy: 0.9300 - dice_coef: 0.84 - ETA: 12s - loss: 0.1530 - binary_accuracy: 0.9295 - dice_coef: 0.84 - ETA: 13s - loss: 0.1564 - binary_accuracy: 0.9311 - dice_coef: 0.84 - ETA: 12s - loss: 0.1696 - binary_accuracy: 0.9316 - dice_coef: 0.83 - ETA: 12s - loss: 0.1670 - binary_accuracy: 0.9297 - dice_coef: 0.83 - ETA: 12s - loss: 0.1665 - binary_accuracy: 0.9290 - dice_coef: 0.83 - ETA: 12s - loss: 0.1657 - binary_accuracy: 0.9276 - dice_coef: 0.83 - ETA: 12s - loss: 0.1625 - binary_accuracy: 0.9288 - dice_coef: 0.83 - ETA: 12s - loss: 0.1647 - binary_accuracy: 0.9260 - dice_coef: 0.83 - ETA: 11s - loss: 0.1640 - binary_accuracy: 0.9264 - dice_coef: 0.83 - ETA: 11s - loss: 0.1664 - binary_accuracy: 0.92

100/100 [==============================] - ETA: 13s - loss: 0.1678 - binary_accuracy: 0.9434 - dice_coef: 0.83 - ETA: 13s - loss: 0.1763 - binary_accuracy: 0.9446 - dice_coef: 0.82 - ETA: 13s - loss: 0.1770 - binary_accuracy: 0.9365 - dice_coef: 0.82 - ETA: 13s - loss: 0.1743 - binary_accuracy: 0.9330 - dice_coef: 0.82 - ETA: 13s - loss: 0.1720 - binary_accuracy: 0.9312 - dice_coef: 0.82 - ETA: 12s - loss: 0.1711 - binary_accuracy: 0.9309 - dice_coef: 0.83 - ETA: 12s - loss: 0.1683 - binary_accuracy: 0.9355 - dice_coef: 0.83 - ETA: 12s - loss: 0.1694 - binary_accuracy: 0.9350 - dice_coef: 0.83 - ETA: 12s - loss: 0.1679 - binary_accuracy: 0.9355 - dice_coef: 0.83 - ETA: 12s - loss: 0.1670 - binary_accuracy: 0.9365 - dice_coef: 0.83 - ETA: 12s - loss: 0.1688 - binary_accuracy: 0.9375 - dice_coef: 0.83 - ETA: 11s - loss: 0.1684 - binary_accuracy: 0.9352 - dice_coef: 0.83 - ETA: 11s - loss: 0.1682 - binary_accuracy: 0.9325 - dice_coef: 0.83 - ETA: 11s - loss: 0.1683 - binary_accuracy: 0.93

In [13]:
#from perception.infers.segmention_infer_prob import SegmentionInfer
from perception.infers.segmention_infer import SegmentionInferprob,SegmentionInfer,SegmentionInferbinary,SegmentionInferbinaryprob,SegmentionInferMulticlass

#from perception.infers.segmention_infer_STARE import SegmentionInfer

from perception.metric.segmention_metric import *
from configs.utils.config_utils import process_config


repredict=True

def main_test():
    print('[INFO] Reading Configs...')
    config = None

    try:
        config = process_config('./configs/segmention_config.json')
        config.G = 0.80
        config.R = 0.20
        config.B = 0.0   
        config.color_space = 'RGB'
        #config.color_space = 'HSV'
        config.multi_proportion =1        
        #config.seg_num = 4
        #config.test_img_path = ".\\experiments\\VesselNet\\dataset\\DRIVE\\validate\\bright\\"
        #config.test_groundtruth_path = ".\\experiments\\VesselNet\\dataset\\DRIVE\\validate\\bright_gt\\"
        #config.test_img_path = ".\\experiments\\VesselNet\\dataset\\DRIVE\\validate\\dark\\"
        #config.test_groundtruth_path = ".\\experiments\\VesselNet\\dataset\\DRIVE\\validate\\dark_gt\\"
        #config.test_prob_path == '.\\experiments\\VesselNet\\test\\probmap\\'
        #config.test_img_path = ".\\experiments\\VesselNet\\dataset\\DRIVE\\train\\semi_validate_img\\"
        config.test_groundtruth_path = ".\\experiments\\VesselNet\\dataset\\DRIVE\\validate\\origin\\"
        if config.color_space == 'RGB':
            config.test_result_path = ".\\experiments\\VesselNet\\dataset\\DRIVE\\validate\\result_demo_RGB\\"
        elif config.color_space == 'HSV':
            config.test_result_path = ".\\experiments\\VesselNet\\dataset\\DRIVE\\validate\\result_demo_HSV\\"
        elif config.color_space == 'LAB':
            config.test_result_path = ".\\experiments\\VesselNet\\dataset\\DRIVE\\validate\\result_demo_LAB\\"
            
        config.patch_height =128
        config.patch_width = 128  
        config.stride_width = 32
        config.stride_height = 32       

            
        if config.multi_proportion :
            config.test_result_path = ".\\experiments\\VesselNet\\dataset\\DRIVE\\validate\\result_demo_{}_{}G_{}R_{}B\\".format(config.patch_height,config.G,config.R,config.B)
        if not os.path.exists(config.test_result_path):
            os.mkdir(config.test_result_path)
        #config.test_img_path = './experiments/VesselNet/dataset/DRIVE/train/origin/'
       # config.test_gt_path = './experiments/VesselNet/dataset/DRIVE/train/img_gt/'
        
        #config = process_config('./configs/segmention_config_CHASEDB1.json')
        #config.train_img_path = ".\\experiments\\VesselNet\\dataset\\CHASEDB1\\train\\"
       # config.train_groundtruth_path =".\\experiments\\VesselNet\\dataset\\train\\"
        #config.val_img_path = ".\\experiments\\VesselNet\\dataset\\CHASEDB1\\validate\\"
        #config.val_groundtruth_path = ".\\experiments\\VesselNet\\dataset\\validate\\groundtruth\\"
      
        #config.test_img_path =".\\experiments\\VesselNet\\dataset\\HRF\\validate\\images\\"
       # config.test_gt_path = '.\\experiments\\VesselNet\\dataset\\HRF\\validate\\gt\\'
        #config.test_result_path='.\\experiments\\VesselNet\\dataset\\HRF\\result\\'
       #config.test_datatype='jpg'
       # config.test_gt_datatype ='tif'
        #config = process_config('./configs/segmention_config_STARE.json')
        #config.test_img_path =".\\experiments\\VesselNet\\dataset\\STARE\\img_bmp\\"
        #config.test_gt_path = '.\\experiments\\VesselNet\\dataset\\STARE\\gt_bmp\\'
        #config.test_result_path='.\\experiments\\VesselNet\\dataset\\STARE\\result\\'
        #config.test_datatype='bmp'
        #config.test_gt_datatype ='bmp'

        #config.test_prob_path = '.\\experiments\\VesselNet\\test\\probmap1\\'
        print(config)
    except Exception as e:
        print('[Exception] Config Error, %s' % e)
        exit(0)

    if repredict==True:
        #config.color_space = 'RGB'
        #config.color_space = 'HSV'
        config.multi_proportion = 1
       # print('[INFO] Predicting...')
        #infer = SegmentionInferprob(config)
        infer = SegmentionInferbinary(config)
        #infer = SegmentionInferbinaryprob(config)
        #infer = SegmentionInfer(config)
        #infer = SegmentionInferMulticlass(config)
        infer.predict()

    print('[INFO] Metric results...')
    gtlist=fileList(config.test_gt_path,'*'+config.test_gt_datatype)
    problist=fileList(config.test_result_path,'*.bmp')
    #print(gtlist)
   # print()
    modelName=['DenseNet-Unet']
    #drawCurve(gtlist,[problist],modelName,'DRIVE',config.checkpoint)

    print('[INFO] Fininshed...')


if __name__ == '__main__':
    main_test()


[INFO] Reading Configs...
B: 0.0
G: 0.8
R: 0.2
batch_size: 25
checkpoint: ./experiments\VesselNet\checkpoint/
color_space: RGB
epochs: 10
exp_name: VesselNet
hdf5_path: ./experiments\VesselNet\hdf5/
height: 584
multi_proportion: 1
patch_height: 128
patch_width: 128
seg_num: 1
stride_height: 32
stride_width: 32
subsample: 500
test_datatype: tif
test_groundtruth_path: .\experiments\VesselNet\dataset\DRIVE\validate\origin\
test_gt_datatype: tif
test_gt_path: ./experiments\VesselNet\test/groundtruth/
test_img_path: ./experiments\VesselNet\test/origin/
test_result_path: .\experiments\VesselNet\dataset\DRIVE\validate\result_demo_128_0.8G_0.2R_0.0B\
total_train: 40
total_val: 20
train_datatype: tif
train_groundtruth_path: ./experiments\VesselNet\dataset/train/groundtruth/
train_gt_datatype: tif
train_img_path: ./experiments\VesselNet\dataset/train/origin/
val_datatype: tif
val_groundtruth_path: ./experiments\VesselNet\dataset/validate/groundtruth/
val_gt_datatype: tif
val_img_path: ./experime

240/240 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 1s 5ms/step
N_patches_h: 16
N_patches_w: 15
N_patches_img: 240
According to the dimension inserted, there are 1 full images (of 608x576 each)
240 240
using avg
Color Space: RGB
G_ratio:0.8 R_ratio:0.2 B_ratio:0.0
[Info] Analyze filename... 20_test
new full images shape: 
(1, 608, 576, 1)
240/240 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 1s 5ms/step
N_patches_h: 16
N_patches_w: 15
N_patches_img: 240
According to the dimension inserted, there are 1 full images (of 608x576 each)
240 240
using avg
[INFO] Metric results...
[INFO] Fininshed...


In [22]:
def Metric(mask_path,result_path,gt_path,threshold=128):   
    tp = []
    fn = []
    tn = []
    fp = []
    #target_path = './metric/DRIVE/Final_Result/'
    #if not os.path.exists(target_path):
       # os.mkdir(target_path)
#        
#    if not os.path
    for thresh in tqdm(range(129,130)):
        temp_tp = 0
        temp_tn = 0
        temp_fp = 0
        temp_fn = 0
        AUC = 0

        for name in os.listdir(gt_path):
            gt_name = os.path.join(gt_path,name)
            gt =  np.array(Image.open((gt_name)))
            mask_name = name.replace('manual1.tif','test_mask.tif')
#            mask_name = name.replace('manual1.tif','training_mask.tif')

            mask_name = os.path.join(mask_path,mask_name)
#            print(mask_name)
            
            mask = cv2.imread(mask_name,0)
#            print(mask.shape)
            result_name = name.replace('manual1.tif','test_prob.bmp')
#            print(result_name)
#            result_name = name.replace('manual1.tif','training_prob.bmp')

            result_name_path = os.path.join(result_path,result_name.replace('bmp','tif'))
            result = cv2.imread(result_name_path,0)
            

    
            
#            result1 =result
#            result1[result1>=127]=255
#            result1[result1<127]=0
#            cv2.imwrite(target_path+name,result1)            
            
            prob_img = result
            gt_img = gt
#            numpy_name = result_name.replace('test_prob.bmp','test.npy')
#            prob_numpy = np.load(numpy_name)
            FOV_result = []
            FOV_gt = []
#            total_result += result
#            total_gt += gt
            for i in range(mask.shape[0]):
                for j in range(mask.shape[1]):
                    if mask[i][j] > 0 :
                        FOV_result.append(result[i][j])
                        FOV_gt.append(gt[i][j])
            
            prob_img = np.array(FOV_result)
#            prob_img[prob_img>120]=128
            gt_img = (np.array(FOV_gt)>0 )* 1
#            total_result += result
#            total_gt += gt
            AUC+= roc_auc_score(gt_img,prob_img)
#        print(AUC/20)
            new_result = (np.array(prob_img)>thresh)*1
            new_gt = (np.array(gt_img)>0)*1
#        
            temp_tp += np.sum(new_result * new_gt)
            temp_tn += np.sum((1-new_result) * (1-new_gt))
            temp_fn += np.sum((1-new_result) *new_gt)
            temp_fp += np.sum(new_result * (1-new_gt))
#        AUC+= roc_auc_score(total_gt/20,total_result/20)2
        print(AUC/20)            
#        print(AUC)
        tp.append(temp_tp)
        tn.append(temp_tn)
        fp.append(temp_fp)
        fn.append(temp_fn)
##    print(tp)
#    
    tp = np.asarray(tp).astype('float32')    
    tn = np.asarray(tn).astype('float32')
    fp = np.asarray(fp).astype('float32')
    fn = np.asarray(fn).astype('float32')
##    print(tp)
    sen =  tp / (tp+fn)
    spe =  tn / (tn+fp)
    acc = (tp+tn)/(tp+fn+fp+tn)
    threshold = np.argmax(acc)

# 
    print('sen: {} spe: {} acc: {} '.format(sen[threshold],spe[threshold],acc[threshold]))


In [26]:

config = process_config('./configs/segmention_config.json')
config.color_space = 'RGB'
config.multi_proportion = 1        

config.patch_height = 128
config.G = 0.65
config.R = 0.35
config.B = 0.0 
mask_path = './metric/DRIVE/mask/'
gt_path = './metric/DRIVE/groundtruth/'
if config.color_space == 'RGB':
    config.test_result_path = ".\\experiments\\VesselNet\\dataset\\DRIVE\\validate\\result_demo_RGB\\"
elif config.color_space == 'HSV':
    config.test_result_path = ".\\experiments\\VesselNet\\dataset\\DRIVE\\validate\\result_demo_HSV\\"
elif config.color_space == 'LAB':
    config.test_result_path = ".\\experiments\\VesselNet\\dataset\\DRIVE\\validate\\result_demo_LAB\\"

if config.multi_proportion :
    config.test_result_path = ".\\experiments\\VesselNet\\dataset\\DRIVE\\validate\\result_demo_{}_{}G_{}R_{}B\\".format(config.patch_height,config.G,config.R,config.B)
if not os.path.exists(config.test_result_path):
    os.mkdir(config.test_result_path)
Metric(mask_path,config.test_result_path,gt_path,threshold=0)    

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

0.9800723475824837


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:12<00:00, 12.38s/it]


sen: 0.7950364351272583 spe: 0.9798888564109802 acc: 0.9563594460487366 
